In [ ]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
music_list = []
df1= pd.DataFrame()
for year in range(1960,2006):
    try:
        df_list = pd.read_html("https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_"+ str(year), header=0)
        df4= df_list[0]
        try:
            df4["Title"]=  df4["Title"].apply(lambda x: json.loads(x))
        except:
            pass
        df4["Year"] = year
        del df4["№"]
        for index, row in df4.iterrows():
            music_list.append(row.values.tolist())
    except:
        pass


# For 2006 to 2017
import requests
from bs4 import BeautifulSoup
for year in range(2006,2018):
    url = "https://www.billboard.com/charts/year-end/"+ str(year)+"/hot-100-songs"
    response = requests.get(url)
    results_page = BeautifulSoup(response.content,'lxml')
    # print(results_page.prettify())
    article_tags = results_page.find_all('article', class_= "ye-chart-item")
    for tag in article_tags:
        artist =tag.find('div', class_= "ye-chart-item__artist").get_text().strip()
        track  = tag.find('div', class_= "ye-chart-item__title").get_text().strip()
        music_list.append([track,artist, year])


In [ ]:
# df = pd.DataFrame(music_list)
# df.tail(50)

In [ ]:
#for 2018
import requests
from bs4 import BeautifulSoup
url = "https://www.billboard.com/charts/hot-100"
response = requests.get(url)
results_page = BeautifulSoup(response.content,'lxml')
# print(results_page.prettify())
div_tags = results_page.find_all('div', class_= "chart-list-item")
for div in div_tags:
    artist= div.get("data-artist")
    track = div.get("data-title")
        

In [ ]:
# !pip3 install spotipy


In [ ]:
import spotipy
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id="a31e2c1446fd4e6aa11476d8532fc939",client_secret="b52f884f9d4f464eba58daf0d5180ea4")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
from collections import OrderedDict
def get_spotify_features(track, artist):
    #Search for Spofity song ID 
    songs=sp.search(q='track:'+track+' '+'artist:'+artist+'*' , type='track')
    items = songs['tracks']['items']
#     print(items)
    if len(items) ==0:
        return([0]*18)
    else:
        track = items[0]
        song_id = str(track["id"])
        #Use ID to get Song features
        track_features=sp.audio_features(song_id)
        if len(track_features[0]) <18:
            return([0]*18)
        else:
            features = np.array(track_features)[0]
            #Order Dictionary
#             print(len(track_features[0]))
            features = OrderedDict(features)
#             return (features)
            #Get Dictionary values
            feature_values = features.values()
            return(feature_values)
 ## get_spotify_features("Sicko Mode", "Travis Scott")    
audio_features = []
for i in range(0,len(music_list)):
    try:
        year = music_list[i][2]
        track = music_list[i][0]
        artist= music_list[i][1]
        get_spotify_features(track,artist)
        danceability, energy, key, loudness, mode,speechiness,acousticness, instrumentalness, liveness, valence, tempo, type_, id_,uri, track_href,analysis_url,duration_ms, time_signature = get_spotify_features(track, artist)
        audio_features.append((year,artist,track,danceability, energy, key, loudness, mode, speechiness,acousticness, instrumentalness, liveness, valence, tempo,duration_ms, time_signature))
    except:
        pass

In [ ]:
import pandas as pd
labels = ["Year", "Artist", "Track", "Danceability","Energy", "Key", "Loudness", "Mode", "Speechiness", "Acousticness",
          "Instrumentalness", "Liveness", "Valence", "Tempo", "Duration_ms", "Time_Signature"]
df =pd.DataFrame(audio_features, columns=labels)
# df.head(50)

In [ ]:
df.to_csv("spotify_project.csv", sep='\t')


In [ ]:

def get_popularity(track, artist):
    import spotipy
    import numpy as np
    from spotipy.oauth2 import SpotifyClientCredentials
    client_credentials_manager = SpotifyClientCredentials(client_id="a31e2c1446fd4e6aa11476d8532fc939",client_secret="b52f884f9d4f464eba58daf0d5180ea4")
    from collections import OrderedDict

    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    artists=sp.search('artist:'+artist+'*' , type='artist')
    try:
        artist_popularity = artists['artists']['items'][0]['popularity']
    except:
        artist_popularity = 0
        
    songs=sp.search(q='track:'+track+' '+'artist:'+artist+'*' , type='track')
    items = songs['tracks']['items']
    if len(items) ==0:
        return([0]*18)
        song_popularity = 0
    else:
        track = items[0]
#         print(track)
        song_id = str(track["id"])
        song_popularity = track['popularity']
    return (song_popularity, artist_popularity)

popularity=[]
for i in range(0,len(music_list)):
    try:
        year = music_list[i][2]
        track = music_list[i][0]
        artist= music_list[i][1]
        song_popularity, artist_popularity= get_popularity(track, artist)
        popularity.append((track, song_popularity, artist_popularity))
        print(i)
    except:
        pass

In [ ]:
import pandas as pd
labels = ["Track", "Track_Popularity", "Artist_Popularity"]
df_pop =pd.DataFrame(popularity, columns=labels)
df_pop

In [ ]:
df_pop.to_csv("spotify_pop_project.csv", sep='\t')